# Azure Privileged Identity Management

[Azure AD Privileged Identity Management (PIM)](https://learn.microsoft.com/en-us/entra/id-governance/privileged-identity-management/pim-configure) is a service that enables you to manage, control, and monitor access to important resources in your organization. These resources include resources in Azure Entra Id such as administrative roles and access to Azure resources.

Privileged Identity Management provides time-based and approval-based role activation to mitigate the risks of excessive, unnecessary, or misused access permissions on resources

## This notebook

This notebook will walk you through the following steps:
- Configure the role settings like notifications, role assignment duration, and approval workflow
- Assign eligible roles to users

# Prerequisites

To connect to the Microsoft Graph Api you need to have an app registration in Microsoft Entra Id with the following API permissions for Microsoft Graph:
- RoleManagement.ReadWrite.Directory

### Set required variables

This cell sets the variables that are required to run the notebook. The variables are used in the rest of the notebook. 

- The `approversEntraIDGroup` is used in this notebook to identify the Azure Entra ID Group that contains the approvers for the PIM role assignments using the Object Id.
- The `eligiblePrincipalId` is used in this notebook to identify the [Azure Entra ID group](https://learn.microsoft.com/en-us/entra/identity/role-based-access-control/groups-create-eligible?tabs=ms-powershell) or user who is able to request a role assignment using the Object Id..
- The `privilegedRoles` denotes roles that require approval before they can become active.


In [13]:
var tenantId = "";
var clientId = "";
var approversEntraIDGroup = "";
var approversEntraIDGroupName = "PIM Approvers";
var eligiblePrincipalId = "";  
var privilegedRoles = new[] { "Application Administrator", "Application Developer", "Authentication Administrator", "Authentication Extensibility Administrator", "B2C IEF Keyset Administrator", "Cloud Application Administrator", "Cloud Device Administrator", "Conditional Access Administrator", "Directory Writers", "Global Administrator", "Global Reader", "Helpdesk Administrator", "Hybrid Identity Administrator", "Intune Administrator", "Password Administrator", "Privileged Authentication Administrator", "Privileged Role Administrator", "Security Administrator", "Security Operator", "Security Reader", "User Administrator" };

### Set up the Microsoft Graph SDK

The [Microsoft Graph SDK for .NET](https://learn.microsoft.com/en-us/graph/sdks/sdks-overview) allows for inspecting, creating and updating of Azure Entra ID Resources. The client is used to update PIM role settings and assign roles to groups.

In [ ]:
#r "nuget:Azure.Identity"
#r "nuget:Microsoft.Graph"

using Azure.Identity;
using Microsoft.Graph;
using Microsoft.Graph.Models;

var scopes = new[] { "RoleManagement.ReadWrite.Directory" };
var graphOptions = new InteractiveBrowserCredentialOptions
{
	TenantId = tenantId,
	ClientId = clientId,
	AuthorityHost = AzureAuthorityHosts.AzurePublicCloud,
	RedirectUri = new Uri("http://localhost")
};

var interactiveCredential = new InteractiveBrowserCredential(graphOptions);
var graphClient = new GraphServiceClient(interactiveCredential, scopes);


Build list of roles and their corresponding role policy. Approval is required when the role is in the `privilegedRoles` list.

In [ ]:
var policyApprovalRequired = new Dictionary<string, bool>();
var assignments = await graphClient.Policies.RoleManagementPolicyAssignments.GetAsync((requestConfiguration) =>
						{
							requestConfiguration.QueryParameters.Filter = "scopeId eq '/' and scopeType eq 'DirectoryRole'";
						});
foreach (var assignment in assignments.Value)
{
	var role = (await graphClient.DirectoryRoleTemplates[assignment.RoleDefinitionId].GetAsync());
	policyApprovalRequired.Add(assignment.PolicyId, privilegedRoles.Contains(role.DisplayName));
}


### Modify default PIM role settings

This code modifies the default [PIM role settings](https://learn.microsoft.com/en-us/entra/id-governance/privileged-identity-management/pim-resource-roles-configure-role-settings#role-settings). The default settings are modified to allow permanent eligible assignments and sets an Azure Entra ID Group with approvers for the role assignments.

In [ ]:
var policies = await graphClient.Policies.RoleManagementPolicies.GetAsync((requestConfiguration) =>
{
	requestConfiguration.QueryParameters.Filter = "scopeId eq '/' and scopeType eq 'DirectoryRole'";
	requestConfiguration.QueryParameters.Expand = new string []{ "rules" };
});

foreach (var policy in policies.Value)
{
    foreach (var rule in policy.Rules)
    {
        switch (rule)
        {
            case UnifiedRoleManagementPolicyApprovalRule approvalRule:
                if (rule.Id != "Approval_EndUser_Assignment")
                    break;

                approvalRule.Setting.IsApprovalRequired = policyApprovalRequired.GetValueOrDefault(policy.Id!);

                var stage = approvalRule.Setting.ApprovalStages.First();
                if (!approvalRule.Setting.IsApprovalRequired.GetValueOrDefault())
                    stage.PrimaryApprovers.Clear();

                if (stage.PrimaryApprovers.OfType<GroupMembers>().Any(gm => gm.GroupId == approversEntraIDGroup) || !approvalRule.Setting.IsApprovalRequired.GetValueOrDefault())
                    break;

                stage.PrimaryApprovers.Add(new GroupMembers
                {
                    Description = approversEntraIDGroupName,
                    GroupId = approversEntraIDGroup,
                });
                break;
            case UnifiedRoleManagementPolicyNotificationRule notificationRule:
                switch (rule.Id)
                {
                    case "Notification_Admin_Admin_Eligibility":
                    case "Notification_Admin_Admin_Assignment":
                    case "Notification_Admin_EndUser_Assignment":
                    case "Notification_Requestor_Admin_Eligibility":
                    case "Notification_Requestor_Admin_Assignment":
                        notificationRule.IsDefaultRecipientsEnabled = false;
                        break;
                    default:
                        notificationRule.IsDefaultRecipientsEnabled = true;
                        break;
                }
                break;
            case UnifiedRoleManagementPolicyExpirationRule expirationRule:
                expirationRule.IsExpirationRequired = rule.Id != "Expiration_Admin_Eligibility";
                break;
        }

        try
    	{
	    	var result = await graphClient.Policies.RoleManagementPolicies[policy.Id].Rules[rule.Id].PatchAsync(rule);
	    }
	    catch(Exception ex)
	    {
            policy.Display();
		    Console.WriteLine($"failed to update rule '{rule.Id}' for policy {policy.DisplayName}: {ex.Message}");
	    }
    }
}

### Assing role eligibility to Azure Entra ID Group

Before a user can request a role assignment, the user must be eligible for the role. This code assigns the Azure Entra ID Group to the role eligibility. The group is used to identify the users that are eligible for the role.

In [ ]:
var directoryRoles = await graphClient.DirectoryRoleTemplates.GetAsync();

foreach(var directoryRole in directoryRoles.Value)
{
	var requestBody = new UnifiedRoleEligibilityScheduleRequest
	{
		Action = UnifiedRoleScheduleRequestActions.AdminAssign,
		RoleDefinitionId = directoryRole.Id,
		DirectoryScopeId = "/",
		PrincipalId = eligiblePrincipalId,
		ScheduleInfo = new RequestSchedule
		{
			Expiration = new ExpirationPattern
			{
				Type = ExpirationPatternType.NoExpiration
			}
		}
	};
	try
	{
		var result = await graphClient.RoleManagement.Directory.RoleEligibilityScheduleRequests.PostAsync(requestBody);
	}
	catch(Exception ex)
	{
		Console.WriteLine($"failed to assign role '{directoryRole.DisplayName}' ({directoryRole.Id}) to principal {eligiblePrincipalId}: {ex.Message}");
	}
};